In [18]:
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict

#Plotly
from plotly import graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected = True)



#Topic Modeling
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.wrappers.ldamallet import malletmodel2ldamodel

# Estudio de la energía de coherencia

In [19]:
energy_mallet_dict = pd.read_pickle('../Modelos/Coherence_Energy/energy_mallet.pickle')
energy_lda_dict = pd.read_pickle('../Modelos/Coherence_Energy/energy_lda.pickle')

In [24]:
energy_lda

defaultdict(None,
            {'Todos': <pandas.core.indexing._iLocIndexer at 0x7f73ce3813b8>})

In [29]:
# Estudio de la energía Gensim
iterable = energy_lda_dict.items()
energy_lda=defaultdict()
for stock, energy_dict in iterable:
    df = pd.DataFrame.from_dict(energy_dict, orient='index', columns=['energia'])
    df = df.sort_values('energia', axis=0, ascending=False)
    energy_lda[stock] = df
energy_lda_df = pd.concat(energy_lda)
energy_lda_df

energia
Todos Todos_topics7_iters200.model  176.294319
      Todos_topics7_iters500.model  174.183141
      Todos_topics7_iters350.model  174.183141
      Todos_topics9_iters500.model  172.268531
      Todos_topics9_iters350.model  172.268531
      Todos_topics9_iters200.model  171.874582
      Todos_topics5_iters200.model  168.609293
      Todos_topics5_iters500.model  168.499138
      Todos_topics5_iters350.model  168.427163
      Todos_topics6_iters200.model  164.158944
      Todos_topics6_iters350.model  161.513144
      Todos_topics6_iters500.model  161.421337
      Todos_topics4_iters200.model  153.535972
      Todos_topics3_iters200.model  151.822162
      Todos_topics4_iters350.model  149.971832
      Todos_topics4_iters500.model  149.944641
      Todos_topics8_iters200.model  147.310388
      Todos_topics8_iters350.model  146.317083
      Todos_topics8_iters500.model  145.659277
      Todos_topics3_iters500.model  140.708302
      Todos_topics3_iters350.model  140.708302
      Todos_topics2_iters200.model  136.708761
      Todos_topics2_iters350.model  129.619089
      Todos_topics2_iters500.model  129.564709

In [30]:
# Estudio de la energía mallet
iterable = energy_mallet_dict.items()
energy_mallet = defaultdict()
for stock, energy_dict in iterable:
    df = pd.DataFrame.from_dict(energy_dict, orient='index', columns=['energia'])
    df = df.sort_values('energia', axis=0, ascending=False)
    df = df
    energy_mallet[stock] = df
energy_mallet_df = pd.concat(energy_mallet)
energy_mallet_df

energia
Todos Todos_topics5_iters350.model  160.016534
      Todos_topics5_iters500.model  160.016534
      Todos_topics5_iters200.model  160.015185
      Todos_topics9_iters200.model  159.934542
      Todos_topics8_iters350.model  155.505885
      Todos_topics8_iters200.model  155.502884
      Todos_topics8_iters500.model  155.502884
      Todos_topics9_iters500.model  148.206594
      Todos_topics9_iters350.model  147.547019
      Todos_topics6_iters200.model  137.413193
      Todos_topics6_iters350.model  134.488291
      Todos_topics2_iters350.model  134.400876
      Todos_topics2_iters500.model  134.400876
      Todos_topics2_iters200.model  134.400876
      Todos_topics6_iters500.model  133.442275
      Todos_topics3_iters500.model  130.668227
      Todos_topics3_iters350.model  130.668227
      Todos_topics3_iters200.model  130.668227
      Todos_topics7_iters500.model  129.520758
      Todos_topics7_iters200.model  129.440070
      Todos_topics7_iters350.model  128.259298
      Todos_topics4_iters500.model  124.569410
      Todos_topics4_iters350.model  124.569410
      Todos_topics4_iters200.model  123.821561

In [31]:
stock = 'Todos'

def graph_energies(lda_df,mallet_df,stock):
    lda_stock = lda_df.loc[stock]
    mallet_stock = mallet_df.loc[stock]
    traces = list()
    iterable_lda = lda_stock.iterrows()
    for item in iterable_lda:
        modelo = item[0]
        modelo = modelo.strip('.model')
        topics = modelo.split('_')[-2]
        iters = modelo.split('_')[-1]
        energy = item[1].energia
        trace = go.Bar({
                        'x': [topics],
                        'y': [energy],
                        'name': f'gensim_{topics}_{iters}',
                        'marker': {'color': '#086788'}
        })
        traces.append(trace)
    iterable_mallet = mallet_stock.iterrows()
    for item in iterable_mallet:
        modelo = item[0]
        modelo = modelo.strip('.model')
        topics = modelo.split('_')[-2]
        iters = modelo.split('_')[-1]
        energy = item[1].energia
        trace = go.Bar({
                        'x': [topics],
                        'y': [energy],
                        'name': f'mallet_{topics}_{iters}',
                        'marker': {'color': '#AE3922'}
        })
        traces.append(trace)
    return traces

layout = go.Layout({'title': stock})

traces = graph_energies(energy_lda_df,energy_mallet_df,stock)

fig = go.Figure(traces, layout)

iplot(fig)

# Visualización pyLDAvis

In [32]:
import pyLDAvis.gensim as vis
import pyLDAvis
from os import listdir
pyLDAvis.enable_notebook()

In [33]:
lista_gensim = listdir('../Modelos/Gensim')
lista_mallet = listdir('../Modelos/Mallet')
mds = ['PCoA', 'tsne']

In [34]:
def vis_gensim(model, mds):
    tuple_ = pickle.load(open(f'../Modelos/Gensim/{model}','rb'))
    model = tuple_[0]
    corpus = tuple_[3]
    vocab = tuple_[2]
    topic_data=vis.prepare(model,corpus,vocab, mds=mds)
    pyLDAvis.display(topic_data)
    return pyLDAvis.display(topic_data)

In [35]:
def vis_mallet(model, mds):
    tuple_ = pickle.load(open(f'../Modelos/Mallet/{model}','rb'))
    model = tuple_[0]
    model = malletmodel2ldamodel(model)
    corpus = tuple_[3]
    vocab = tuple_[2]
    topic_data=vis.prepare(model,corpus,vocab, mds=mds)
    return pyLDAvis.display(topic_data)

In [36]:
from ipywidgets import *

In [37]:
widget_gensim = interactive(vis_gensim,{'manual':True},model=lista_gensim, mds=mds)
widget_mallet = interactive(vis_gensim,{'manual':True},model=lista_gensim, mds=mds, )

In [38]:
widget_gensim

interactive(children=(Dropdown(description='model', options=('Todos_topics4_iters350.model', 'Todos_topics9_it…

In [46]:
widget_gensim.result

In [43]:
widget_mallet

interactive(children=(Dropdown(description='model', index=16, options=('Todos_topics4_iters350.model', 'Todos_…

In [45]:
widget_mallet.result